In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
BASE_URL = "https://raw.githubusercontent.com/SurakshitKapoor/Urban-Mobility-Analytics-ML-Platform/main/csv_files/"


In [3]:
fact_trips = pd.read_csv(BASE_URL + "fact_trips.csv")
monthly_target_trips = pd.read_csv(BASE_URL + "monthly_target_trips.csv")
monthly_target_new_passengers = pd.read_csv(BASE_URL + "monthly_target_new_passengers.csv")
fact_passenger_summary = pd.read_csv(BASE_URL + "fact_passenger_summary.csv")
dim_repeat_trip_distribution = pd.read_csv(BASE_URL + "dim_repeat_trip_distribution.csv")
dim_date = pd.read_csv(BASE_URL + "dim_date.csv")
dim_city = pd.read_csv(BASE_URL + "dim_city.csv")
city_target_passenger_rating = pd.read_csv(BASE_URL + "city_target_passenger_rating.csv")


In [4]:
# fact_trips.info()
fact_trips['date'] = pd.to_datetime(fact_trips['date'] , dayfirst=True)

In [6]:
# monthly_target_trips
monthly_target_trips['month'] = pd.to_datetime(monthly_target_trips['month'], dayfirst=True )


In [7]:
monthly_target_trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   month               60 non-null     datetime64[ns]
 1   city_id             60 non-null     object        
 2   total_target_trips  60 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.5+ KB


In [8]:
ans = monthly_target_trips['month'].dt.month.unique()
ans

array([3, 5, 4, 2, 1, 6], dtype=int32)

In [9]:
monthly_target_new_passengers['month'] = pd.to_datetime( monthly_target_new_passengers['month'], dayfirst=True )

In [10]:
fact_passenger_summary['month'] = pd.to_datetime(fact_passenger_summary['month'], dayfirst=True )


In [11]:
dim_repeat_trip_distribution['month'] = pd.to_datetime(dim_repeat_trip_distribution['month'], dayfirst=True)


In [12]:
# dim_date.info()
dim_date['date'] = pd.to_datetime(dim_date['date'], dayfirst=True)
dim_date['start_of_month'] = pd.to_datetime(dim_date['start_of_month'], dayfirst=True)

In [13]:
print(dim_date.date.dt.month.unique())
print(dim_date.date.dt.day.unique())

[1 2 3 4 5 6]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]


In [14]:
# dim_city

In [15]:
# city_target_passenger_rating  

## Primary Questions

In [16]:
# 1.

In [17]:
# trips per city
trips_by_city = (
    fact_trips
    .groupby("city_id")
    .size()
    .reset_index(name="total_trips")
    .merge(dim_city, on="city_id", how="left")
)

top_3_cities = trips_by_city.sort_values("total_trips", ascending=False).head(3)
bottom_3_cities = trips_by_city.sort_values("total_trips").head(3)


In [18]:
top_3_cities

,city_id,total_trips,city_name
7,RJ01,76888,Jaipur
9,UP01,64299,Lucknow
2,GJ01,54843,Surat


In [19]:
bottom_3_cities

,city_id,total_trips,city_name
4,KA01,16238,Mysore
8,TN01,21104,Coimbatore
0,AP01,28366,Visakhapatnam


2. Average Fare per Trip vs Average Distance (by City)

In [20]:
fare_distance_city = (
    fact_trips
    .groupby("city_id")
    .agg(
        avg_fare_per_trip=("fare_amount", "mean"),
        avg_distance_km=("distance_travelled(km)", "mean")
    )
    .reset_index()
    .merge(dim_city, on="city_id", how="left")
)

fare_distance_city

,city_id,avg_fare_per_trip,avg_distance_km,city_name
0,AP01,282.672284,22.553938,Visakhapatnam
1,CH01,283.686950,23.518714,Chandigarh
2,GJ01,117.272925,10.997247,Surat
3,GJ02,118.566165,11.517736,Vadodara
4,KA01,249.707168,16.496921,Mysore
5,KL01,335.245079,24.065461,Kochi
6,MP01,179.838609,16.502473,Indore
7,RJ01,483.918128,30.023125,Jaipur
8,TN01,166.982183,14.979198,Coimbatore
9,UP01,147.180376,12.512963,Lucknow


In [21]:
# optional: pricing efficiency
fare_distance_city["fare_per_km"] = (
    fare_distance_city["avg_fare_per_trip"] /
    fare_distance_city["avg_distance_km"]
)

highest_fare_city = fare_distance_city.sort_values("avg_fare_per_trip", ascending=False).head(1)
lowest_fare_city = fare_distance_city.sort_values("avg_fare_per_trip").head(1)

print("highest_fare_city:\n", highest_fare_city)
print()
print("*"*100)
print()
print("lowest_fare_city:\n", lowest_fare_city)

highest_fare_city:
   city_id  avg_fare_per_trip  avg_distance_km city_name  fare_per_km
7    RJ01         483.918128        30.023125    Jaipur     16.11818

****************************************************************************************************

lowest_fare_city:
   city_id  avg_fare_per_trip  avg_distance_km city_name  fare_per_km
2    GJ01         117.272925        10.997247     Surat    10.663844


3. Average Ratings by City & Passenger Type

In [22]:
ratings_city_passenger = (
    fact_trips
    .groupby(["city_id", "passenger_type"])
    .agg(
        avg_passenger_rating=("passenger_rating", "mean"),
        avg_driver_rating=("driver_rating", "mean")
    )
    .reset_index()
    .merge(dim_city, on="city_id", how="left")
)

# print("average ratings by city and passenger types:\n")
# print(ratings_city_passenger)

In [23]:
ratings_wide = (
    ratings_city_passenger
    .pivot(
        index=["city_id", "city_name"],
        columns="passenger_type",
        values=["avg_passenger_rating", "avg_driver_rating"]
    )
    .reset_index()
)

In [24]:
ratings_wide

city_id      city_name avg_passenger_rating            \
passenger_type                                         new  repeated   
0                 AP01  Visakhapatnam             8.976151  7.989628   
1                 CH01     Chandigarh             8.489158  7.493798   
2                 GJ01          Surat             7.984173  5.995511   
3                 GJ02       Vadodara             7.979263  5.978629   
4                 KA01         Mysore             8.982964  7.978495   
5                 KL01          Kochi             8.987394  8.003665   
6                 MP01         Indore             8.485837  7.473961   
7                 RJ01         Jaipur             8.985018  7.991042   
8                 TN01     Coimbatore             8.485788  7.475457   
9                 UP01        Lucknow             7.977429  5.985741   

               avg_driver_rating            
passenger_type               new  repeated  
0                       8.979995  8.992701  
1                       7.992120  7.472824  
2                       6.994925  6.479441  
3                       7.004147  6.481072  
4                       8.982878  8.965767  
5                       8.985350  8.989830  
6                       7.970800  7.477404  
7                       8.988246  8.984790  
8                       7.990604  7.480778  
9                       6.990406  6.491663

4. Peak & Low Demand Months by City

In [25]:
# add month to trips
fact_trips["month"] = pd.to_datetime(fact_trips["date"]).dt.to_period("M")

monthly_trips = (
    fact_trips
    .groupby(["city_id", "month"])
    .size()
    .reset_index(name="total_trips")
    .merge(dim_city, on="city_id", how="left")
)

monthly_trips



,city_id,month,total_trips,city_name
0,AP01,2024-01,4468,Visakhapatnam
1,AP01,2024-02,4793,Visakhapatnam
2,AP01,2024-03,4877,Visakhapatnam
3,AP01,2024-04,4938,Visakhapatnam
4,AP01,2024-05,4812,Visakhapatnam
5,AP01,2024-06,4478,Visakhapatnam
6,CH01,2024-01,6810,Chandigarh
7,CH01,2024-02,7387,Chandigarh
8,CH01,2024-03,6569,Chandigarh
9,CH01,2024-04,5566,Chandigarh


In [26]:
peak_low_months = (
    monthly_trips
    .sort_values("total_trips")
    .groupby("city_id")
    .agg(
        low_demand_month=("month", "first"),
        peak_demand_month=("month", "last")
    )
    .reset_index()
    .merge(dim_city, on="city_id", how="left")
)

print("***Low and High demand months per city:***\n")
peak_low_months

***Low and High demand months per city:***



,city_id,low_demand_month,peak_demand_month,city_name
0,AP01,2024-01,2024-04,Visakhapatnam
1,CH01,2024-04,2024-02,Chandigarh
2,GJ01,2024-01,2024-04,Surat
3,GJ02,2024-06,2024-04,Vadodara
4,KA01,2024-01,2024-05,Mysore
5,KL01,2024-06,2024-05,Kochi
6,MP01,2024-06,2024-05,Indore
7,RJ01,2024-06,2024-02,Jaipur
8,TN01,2024-06,2024-03,Coimbatore
9,UP01,2024-05,2024-02,Lucknow


5. Weekend vs Weekday Trip Demand by City

In [27]:
trips_day_type = (
    fact_trips
    .merge(dim_date, on="date", how="left")
    .groupby(["city_id", "day_type"])
    .size()
    .reset_index(name="total_trips")
    .merge(dim_city, on="city_id", how="left")
)

# trips_day_type

# pivot for clean view
trips_day_type_wide = (
    trips_day_type
    .pivot(index=["city_id", "city_name"], columns="day_type", values="total_trips")
    .reset_index()
)

print("Weekday and Weekend trips by cities:\n")
trips_day_type_wide

Weekday and Weekend trips by cities:



day_type,city_id,city_name,Weekday,Weekend
0,AP01,Visakhapatnam,15100,13266
1,CH01,Chandigarh,19914,19067
2,GJ01,Surat,37793,17050
3,GJ02,Vadodara,20310,11716
4,KA01,Mysore,6424,9814
5,KL01,Kochi,22915,27787
6,MP01,Indore,21198,21258
7,RJ01,Jaipur,32491,44397
8,TN01,Coimbatore,12576,8528
9,UP01,Lucknow,49617,14682


6. Repeat Passenger Frequency & City Contribution

In [28]:
repeat_freq = (
    dim_repeat_trip_distribution
    .merge(dim_city, on="city_id", how="left")
)

repeat_freq

,month,city_id,trip_count,repeat_passenger_count,city_name
0,2024-01-01,AP01,10-Trips,7,Visakhapatnam
1,2024-01-01,AP01,2-Trips,352,Visakhapatnam
2,2024-01-01,AP01,3-Trips,158,Visakhapatnam
3,2024-01-01,AP01,4-Trips,53,Visakhapatnam
4,2024-01-01,AP01,5-Trips,38,Visakhapatnam
...,...,...,...,...,...
535,2024-06-01,UP01,5-Trips,272,Lucknow
536,2024-06-01,UP01,6-Trips,272,Lucknow
537,2024-06-01,UP01,7-Trips,246,Lucknow
538,2024-06-01,UP01,8-Trips,83,Lucknow


7. Monthly Target Achievement (Trips, New Passengers, Ratings)

In [29]:
# # ---- Actual trips per city and their months
# ---- Join trip targets
trip_target_perf = (
    actual_trips
    .merge(monthly_target_trips, on=["city_id", "month"], how="left")
)
fact_trips["month"] = pd.to_datetime(fact_trips["date"]).dt.to_period("M").astype(str)

actual_trips = (
    fact_trips
    .groupby(["city_id", "month"])
    .size()
    .reset_index(name="actual_trips")
)

# actual_trips

# ---- Join trip targets
monthly_target_trips["month_year"] = pd.to_datetime(monthly_target_trips["month"]).dt.to_period("M").astype(str)

trip_target_perf = (
    actual_trips
    .merge(monthly_target_trips, on=["city_id", "month_year"], how="left")
)

trip_target_perf

NameError: name 'actual_trips' is not defined

In [31]:
# ---- Rating vs target
rating_perf = (
    fact_trips
    .groupby(["city_id"])
    .agg(actual_avg_rating=("passenger_rating", "mean"))
    .reset_index()
    .merge(city_target_passenger_rating, on="city_id", how="left")
)

rating_perf["rating_gap"] = (
    round(rating_perf["actual_avg_rating"] -
    rating_perf["target_avg_passenger_rating"], 2)
)

rating_perf

,city_id,actual_avg_rating,target_avg_passenger_rating,rating_gap
0,AP01,8.432948,8.50,-0.07
1,CH01,7.976604,8.00,-0.02
2,GJ01,6.417081,7.00,-0.58
3,GJ02,6.611253,7.50,-0.89
4,KA01,8.701072,8.50,0.20
5,KL01,8.516193,8.50,0.02
6,MP01,7.828199,8.00,-0.17
7,RJ01,8.583849,8.25,0.33
8,TN01,7.883055,8.25,-0.37
9,UP01,6.489401,7.25,-0.76


In [34]:
# ---- New passenger target achievement
new_passenger_perf = (
    fact_passenger_summary
    .merge(monthly_target_new_passengers, on=["city_id"], how="left")
)

new_passenger_perf["new_passenger_achievement_pct"] = (
    (new_passenger_perf["new_passengers"] - new_passenger_perf["target_new_passengers"])
    / new_passenger_perf["target_new_passengers"]
) * 100

new_passenger_perf

,month_x,city_id,new_passengers,repeat_passengers,total_passengers,month_y,target_new_passengers,new_passenger_achievement_pct
0,2024-01-01,AP01,2513,650,3163,2024-01-01,2500,0.52000
1,2024-01-01,AP01,2513,650,3163,2024-04-01,2000,25.65000
2,2024-01-01,AP01,2513,650,3163,2024-02-01,2500,0.52000
3,2024-01-01,AP01,2513,650,3163,2024-03-01,2500,0.52000
4,2024-01-01,AP01,2513,650,3163,2024-05-01,2000,25.65000
...,...,...,...,...,...,...,...,...
355,2024-06-01,UP01,1971,1727,3698,2024-03-01,3200,-38.40625
356,2024-06-01,UP01,1971,1727,3698,2024-02-01,3200,-38.40625
357,2024-06-01,UP01,1971,1727,3698,2024-06-01,2000,-1.45000
358,2024-06-01,UP01,1971,1727,3698,2024-04-01,2000,-1.45000
